# Flow methods comparison

## Classical methods

In [2]:
import numpy as np
import cv2 as cv

# Fonction pour calculer le flux optique dense
def compute_dense_optical_flow(frame1, frame2):
    # Conversion des images en niveaux de gris
    prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    
    # Calcul du flux optique dense avec la méthode Farneback
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

# Fonction de projection du flux (en pixels)
def project_optical_flow(flow):
    # Calcul de l'amplitude et de l'angle du flux optique
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    return mag, ang

# Fonction pour calculer l'erreur MSE entre deux flux optiques
def compute_mse(flow1, flow2):
    # Calcul de l'erreur quadratique moyenne entre deux flux optiques
    diff = flow1 - flow2
    mse = np.mean(diff ** 2)
    return mse

# Initialisation de la capture vidéo
cap = cv.VideoCapture(cv.samples.findFile("slow_traffic_small.mp4"))

ret, frame1 = cap.read()

# Initialisation du tableau HSV pour la visualisation
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    # Calcul du flux optique dense entre frame1 et frame2
    flow = compute_dense_optical_flow(frame1, frame2)

    # Projeter le flux pour la visualisation (magnitude et angle)
    mag, ang = project_optical_flow(flow)
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

    # Convertir l'image HSV en BGR pour l'affichage
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    
    # Afficher l'image résultante avec le flux optique
    cv.imshow('frame2', bgr)

    # Vérification de l'entrée du clavier pour quitter ou sauvegarder
    k = cv.waitKey(30) & 0xff
    if k == 27:  # Échapper pour quitter
        break
    elif k == ord('s'):  # 's' pour sauvegarder les images
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)

    # Mettre à jour la frame précédente
    frame1 = frame2

cv.destroyAllWindows()

# Pour calculer l'erreur MSE (optionnel)
# Vous pouvez calculer l'erreur entre deux flux consécutifs comme suit :
# mse = compute_mse(flow, previous_flow)


KeyboardInterrupt: 

## Recent methods (Deep Learning) 